In [1]:
import cv2
import numpy as np
import math

In [2]:
im = cv2.imread('im-final.jpg')
mat = np.array([[-0.5115,   -1.4774,  428.8531],
                [-0.0880,   -3.6296,  847.1490],
                [-0.0001,   -0.0054,    1.0000]
               ])
def pointInBirdsEyeView(p):
    p2 = mat.dot(p)
    p2 /= p2[2]
    return p2

In [3]:
im2 = cv2.warpPerspective(im,mat,(1000,1000))
# point = mat.dot(newPoint)
cv2.imwrite('im-res.png',im2)

True

In [4]:
# 左下角点
pLeft = pointInBirdsEyeView([0,426,1])
# 右下角点
pRight = pointInBirdsEyeView([640,426,1])
# 图像上一点 p(x,y,1) x<w, h<h
# 鸟瞰视图坐标 => p' = mat.dot(p)
# p' /= p'[2]
# p' => 坐标x',y'
# 计算 p' 到 直线 pLeft-pRight 的距离d'(m) {k pix => k cm => m} 1 pix = 1 cm
# 标定的底部到相机距离为d(m)
# p到相机的水平距离 D' = d + d'
d = 126
pix2cm = 1

# y = kx + b
k = (pLeft[1] - pRight[1])/(pLeft[0] - pRight[0])
b = pLeft[1] - k*pLeft[0]
# 满足条件的点都应该在线的同一侧
def distance2BottomLine(p):
    # |kx+b-y| / sqrt(k^2 + 1)
    # if the distance < 0, input point is invalid
    # return unit: pix
    return (k*p[0] + b - p[1]) / math.sqrt(k**2 + 1)


def distanceOf2Points(p1,p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

# cm
def distance2Camera(p):
    p = [p[0],p[1],1]
    p = pointInBirdsEyeView(p)
    ds = distance2BottomLine(p)
    if ds < 0:
        return -1
    else:
        return ds * pix2cm + d

In [13]:
distance2Camera([287,264])

368.6898252450313

In [37]:
def computeLineFun(p1, p2):
    k = (p1[1] - p2[1]) / (p1[0] - p2[0])
    b = p1[1] - k*p1[0]
    return (k,b)

def distance2CameraXqq(leftp, rightp):
    p1 = pointInBirdsEyeView([leftp[0],leftp[1],1])
    p2 = pointInBirdsEyeView([rightp[0],rightp[1],1])
    line = computeLineFun(p1, p2)
    print(line)
    x = 273.5925925925924
    y = 672.148148148148
    return -(line[0] * x + line[1] - y) / math.sqrt(line[0] ** 2 + 1)

In [41]:
distance2CameraXqq([0,300],[640,300])

(0.09625449459160203, 387.6582601639544)


-256.96772210179387